In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from paths import mkl_data
import requests # API library

import random
import time
import os

from isoweek import Week

import requests # API library

import numpy as np
import pandas as pd
import json
import re

import pyro
import pyro.distributions as dist
from pyro.contrib.autoguide import AutoDiagonalNormal, AutoMultivariateNormal
from pyro.infer import MCMC, NUTS, HMC, SVI, Trace_ELBO
from pyro.optim import Adam, ClippedAdam
from pyro.infer import Predictive

import torch


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

my_data_path = "df_matches.csv"
df = pd.read_csv(my_data_path, sep=',')
# Drop rows with missing values
# df = df.dropna(subset=['match_date','match_conceded', 'coach1_CR', 'coach2_CR'])

relevant = ["match_id", "division_name", "match_date", "match_time", "match_conceded",
            "team1_coach_id", "team1_race_name", "team2_coach_id", "team2_race_name",
            "team1_score", "team2_score","team1_win","team2_win","mirror_match",
            "coach1_ranking","coach2_ranking","coach1_CR","coach2_CR", "team1_roster_id", "team2_roster_id"]
#irrelevant_columns = set(df.columns) - set(relevant)

irrelevant_columns = {
    'tournament_name', 'week_year', 'cr_diff2', 'team1_cas_bh',
    'group_name', 'team2_foul', 'team2_id', 'coach2_CR_bin',
    'team1_id', 'year', 'week_date', 'week_number', 'replay_id',
    'team1_cas_rip', 'tv_diff', 'team2_block', 'coach1_CR_bin',
    'has_sp', 'team2_value', 'tournament_type', 'team1_value',
    'team2_comp', 'team1_rush', 'team1_foul', 'team2_inducements',
    'tv_diff2', 'tournament_start', 'team2_cas_rip', 'team1_inducements',
    'tv_bin', 'scheduler', 'team1_comp', 'cr_diff2_bin', 'group_id',
    'division_id', 'tournament_id', 'Unnamed: 0', 'team2_cas_si',
    'current_ruleset', 'team2_cas', 'team2_rush', 'team1_block',
    'tournament_end', 'team2_pass', 'team1_cas',
    'CR_diff', 'team1_cas_si', 'team2_cas_bh', 'team1_pass', 'tv_match',
    'tv_bin2', 'tournament_progression'
}



C:\Users\tripl\AppData\Local\Temp\ipykernel_16256\3264193253.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
C:\Users\tripl\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#first load from the competitve league, then filter out the relevant columns
df_matches_competitive = df[df['division_name'] == 'Competitive']
df_matches = df_matches_competitive[relevant]

In [4]:
#Fix types in pandas

# # convert object dtype columns to proper pandas dtypes datetime and numeric
df_matches['match_date'] = pd.to_datetime(df_matches.match_date) # Datetime object

# calculate match score difference
df_matches['team1_win'] = np.sign(df_matches['team1_score'] - df_matches['team2_score'])
df_matches['team2_win'] = np.sign(df_matches['team2_score'] - df_matches['team1_score'])

# mirror match
df_matches['mirror_match'] = 0
df_matches.loc[df_matches['team1_race_name'] == df_matches['team2_race_name'], 'mirror_match'] = 1


# mirror matches
df_matches = df_matches.dropna(subset=['match_date'])

df_matches['week_number'] = df_matches['match_date'].dt.isocalendar().week

# cannot serialize numpy int OR Int64 when writing HDF5 file, so we go for plain int as all NAs are gone now
df_matches['week_number'] = df_matches['week_number'].fillna(0).astype(int)

# add year based on match ISO week
df_matches['year'] = df_matches['match_date'].dt.isocalendar().year.astype(int)

df_matches['week_year'] = df_matches['year'].astype(str) + '-' + df_matches['week_number'].astype(str)

# use a lambda function since isoweek.Week is not vectorized 
df_matches['week_date'] = pd.to_datetime(df_matches.apply(lambda row : Week(int(row["year"]),int(row["week_number"])).monday(),axis=1))

C:\Users\tripl\AppData\Local\Temp\ipykernel_16256\626055672.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_matches['match_date'] = pd.to_datetime(df_matches.match_date) # Datetime object
C:\Users\tripl\AppData\Local\Temp\ipykernel_16256\626055672.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_matches['team1_win'] = np.sign(df_matches['team1_score'] - df_matches['team2_score'])
C:\Users\tripl\AppData\Local\Temp\ipykernel_16256\626055672.py:8: SettingWithCopyWarning: 
A value is trying to be s

In [5]:
df_matches.columns

Index(['match_id', 'division_name', 'match_date', 'match_time',
       'match_conceded', 'team1_coach_id', 'team1_race_name', 'team2_coach_id',
       'team2_race_name', 'team1_score', 'team2_score', 'team1_win',
       'team2_win', 'mirror_match', 'coach1_ranking', 'coach2_ranking',
       'coach1_CR', 'coach2_CR', 'team1_roster_id', 'team2_roster_id',
       'week_number', 'year', 'week_year', 'week_date'],
      dtype='object')

In [6]:
#The selected columns from the competitive dataframe 
selected_columns = [
    'week_date', 
    'match_time',
    'team1_coach_id', 
    'team2_coach_id',
    'team1_win', 
    'team2_win', 
    'team1_race_name', 
    'team2_race_name'
]

#final dataframe 
new_df = df_matches[selected_columns].copy()
new_df['team1_race_name'] = new_df['team1_race_name'].astype('category')
new_df['team2_race_name'] = new_df['team2_race_name'].astype('category')

new_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 95193 entries, 108588 to 258150
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   week_date        95193 non-null  datetime64[ns]
 1   match_time       95193 non-null  object        
 2   team1_coach_id   95193 non-null  int64         
 3   team2_coach_id   95193 non-null  int64         
 4   team1_win        95193 non-null  int64         
 5   team2_win        95193 non-null  int64         
 6   team1_race_name  95193 non-null  category      
 7   team2_race_name  95193 non-null  category      
dtypes: category(2), datetime64[ns](1), int64(4), object(1)
memory usage: 5.3+ MB


In [53]:
#Store the cleaned data in your local directory 
new_df.to_csv('../df_matches_clean.csv', index=False)

#print all values of team1_race_name
races = new_df["team1_race_name"].unique()

num_games_race = np.zeros(len(races))
points_race = np.zeros(len(races))


for i in range(len(races)):
    # Calculate num games by race
    race_df = new_df[(new_df["team1_race_name"] == races[i]) | (new_df["team2_race_name"] == races[i])]
    num_games_race[i] = race_df.shape[0]

    # Calculate points by race
    mask = race_df['team2_race_name'] == races[i]
    race_df.loc[mask, 'team2_win'] *= -1
    race_df.loc[mask, 'team1_win'] *= -1

    points_race[i] = race_df[race_df['team1_win'] == 1].shape[0] + race_df[race_df['team1_win'] == 0].shape[0]/2


for i in range(len(races)):
    print(i, races[i], points_race[i], num_games_race[i], points_race[i]/num_games_race[i])


0 Tomb Kings 3160.0 6108.0 0.5173542894564506
1 Chaos Renegade 2389.5 5683.0 0.420464543374978
2 High Elf 2008.0 4080.0 0.492156862745098
3 Imperial Nobility 2471.0 5729.0 0.4313143655088148
4 Lizardmen 3413.5 7032.0 0.48542377701934014
5 Chaos Dwarf 4583.0 8587.0 0.5337137533480844
6 Black Orc 3911.5 9084.0 0.4305922501100837
7 Chaos Chosen 3086.5 6809.0 0.4532971067704509
8 Amazon 3962.0 6357.0 0.6232499606732735
9 Old World Alliance 1637.0 3583.0 0.45687970974044095
10 Necromantic Horror 4388.0 8640.0 0.5078703703703704
11 Vampire 1143.0 3093.0 0.36954413191076624
12 Goblin 2170.5 5947.0 0.3649739364385404
13 Dark Elf 3848.0 7394.0 0.5204219637543954
14 Human 4717.5 8745.0 0.5394511149228131
15 Orc 7527.0 13037.0 0.5773567538544143
16 Shambling Undead 4724.0 8027.0 0.5885137660396162
17 Wood Elf 2631.5 5497.0 0.47871566308895763
18 Skaven 4317.0 7943.0 0.5434974191111671
19 Underworld Denizens 4872.5 7754.0 0.6283853494970338
20 Dwarf 4440.5 8141.0 0.545448962043975
21 Ogre 2420.0 6

[3160.  2389.5 2008.  2471.  3413.5 4583.  3911.5 3086.5 3962.  1637.
 4388.  1143.  2170.5 3848.  4717.5 7527.  4724.  2631.5 4317.  4872.5
 4440.5 2420.  1905.  4637.5 1379.5 2028.5 2056.5 3752. ]
[ 6108.  5683.  4080.  5729.  7032.  8587.  9084.  6809.  6357.  3583.
  8640.  3093.  5947.  7394.  8745. 13037.  8027.  5497.  7943.  7754.
  8141.  6270.  3967.  8834.  3402.  3766.  4750.  9105.]
[0.51735429 0.42046454 0.49215686 0.43131437 0.48542378 0.53371375
 0.43059225 0.45329711 0.62324996 0.45687971 0.50787037 0.36954413
 0.36497394 0.52042196 0.53945111 0.57735675 0.58851377 0.47871566
 0.54349742 0.62838535 0.54544896 0.38596491 0.48021175 0.52496038
 0.40549677 0.53863516 0.43294737 0.41208127]
